In [39]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

df = pd.read_csv('biased_dataset2.csv')
df_time = pd.read_csv('transactions2.csv', delimiter=';', header=1)
df_time = df_time.dropna()
df_time["date"] = pd.to_datetime(df_time["date"])
df_time['share_of_accepted_transactions'] =df_time['share_of_accepted_transactions'].replace(',','.', regex=True)
df_time['share_of_accepted_transactions'] = df_time['share_of_accepted_transactions'].astype(float)

In [2]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('biased_dataset2.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [114]:
cat_columns = ['Sex', 'ForeignWorker', 'LoanPurpose','CheckingStatus', 'CreditHistory', 'ExistingSavings', 'EmploymentDuration', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone']
num_columns = ['Age', 'LoanDuration', 'InstallmentPercent', 'CurrentResidenceDuration', 'ExistingCreditsCount', 'Dependents']


In [101]:
pd.set_option('display.max_columns', None)
df.head(1)

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,CurrentResidenceDuration,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,3,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk


In [102]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [103]:
idf2 = df_time.interactive()

## (1) Verteilung an Risikofällen pro Spalte

In [104]:
select_cat_distribution_pipeline = pn.widgets.Select(name='Select', options=cat_columns)

In [105]:
cat_distribution_pipeline = (
        idf
        .groupby([select_cat_distribution_pipeline])['Risk']
        .value_counts(normalize=True)
        .mul(100)
        .rename('percent')
        #.reset_index(drop=False)
)
cat_distribution_pipeline

In [106]:
cat_distribution_plot = cat_distribution_pipeline.hvplot.bar(
    title="Verhältnis von Risikofällen für kategorische Spalten",
    ylabel="Prozentualer Anteil",
    stacked=True,
    height=500,
    width=600,
    color=["#1a6b54", "#ba2649"]
)
cat_distribution_plot

In [9]:
# Maybe To-Do: add Radio button for relativ and absolute results. If not possible add two vizualisations
#yaxis_co2 = pn.widgets.RadioButtonGroup(
#    name='Y axis', 
#    options=['Relativ', 'Absolut',],
#    button_type='success'
#)
#yaxis_co2

## (2) Verteilung von Risikofällen für numerische Spalten

In [107]:
df['Risk'] = df['Risk'].replace({'Risk': 1, 'No Risk': 0})

In [108]:
select_num_distribution_pipeline = pn.widgets.Select(name='Select', options=num_columns)
select_num_distribution_pipeline

Select(options=['LoanDuration', ...], value='LoanDuration')

In [109]:
num_distribution_pipeline = (
    idf
    .assign(total=lambda x: x.groupby('Age')['Risk'].transform('count'))
    .assign(NoRisk=lambda x: x['Risk'].eq(1).groupby(x['Age']).transform('sum') / x['total'] )
    .assign(Risk=lambda x: x['Risk'].eq(0).groupby(x['Age']).transform('sum') / x['total'] )
    .loc[:, ['Age', 'total', 'Risk', 'NoRisk']]
    .sort_values('Age')
    .drop_duplicates()
)
num_distribution_pipeline

In [110]:
num_distribution_plot = num_distribution_pipeline.hvplot.line(
    x='Age',
    y=['Risk', 'NoRisk'],
    title="Verhältnis von Risikofällen für numerische Spalten",
    ylabel="Prozentualer Anteil",
    height=500,
    width=600,
    color=["#1a6b54", "#ba2649"],
    line_width=3,
    legend='top'
)
num_distribution_plot

In [113]:
select_num_distribution_pipeline = pn.widgets.Select(name='Select', options=num_columns)

num_distribution_pipeline = (
    idf
    .assign(total=lambda x: x.groupby(select_num_distribution_pipeline.value)['Risk'].transform('count'))
    .assign(NoRisk=lambda x: x['Risk'].eq(1).groupby(x[select_num_distribution_pipeline.value]).transform('sum') / x['total'] )
    .assign(Risk=lambda x: x['Risk'].eq(0).groupby(x[select_num_distribution_pipeline.value]).transform('sum') / x['total'] )
    .loc[:, [select_num_distribution_pipeline, 'total', 'Risk', 'NoRisk']]
    .sort_values(select_num_distribution_pipeline)
    .drop_duplicates()
)
num_distribution_pipeline

In [112]:
num_distribution_plot = num_distribution_pipeline.hvplot.line(
    x=select_num_distribution_pipeline,
    y=['Risk', 'NoRisk'],
    title="Anteil an Risikofällen",
    ylabel="Verteilung an Risikofällen",
    height=500,
    width=600,
    color=["#1a6b54", "#ba2649"],
    line_width=3,
    legend='top'
)
num_distribution_plot

## (3) Anzahl an unterschiedlichen Werten und deren Verteilung für Kategorische Werte

In [16]:
cat_distribution_pipeline2 = (
    idf
    .assign(total=lambda x: x.groupby(select_cat_distribution_pipeline.value)['Risk'].transform('count'))
    .assign(NoRisk=lambda x: x['Risk'].eq(1).groupby(x[select_cat_distribution_pipeline.value]).transform('sum') / x['total'] )
    .assign(Risk=lambda x: x['Risk'].eq(0).groupby(x[select_cat_distribution_pipeline.value]).transform('sum') / x['total'] )
    .loc[:, [select_cat_distribution_pipeline, 'total']]
    .sort_values(select_cat_distribution_pipeline)
    .drop_duplicates()
)
cat_distribution_pipeline2

In [88]:
cat_distribution_plot2 = cat_distribution_pipeline2.hvplot.bar(
    x=select_cat_distribution_pipeline,
    y='total',
    title="Anzahl vorkommender Werte in kategorischen Spalten",
    ylabel="Anzahl",
    height=500,
    width=600,
    legend='top'
)
cat_distribution_plot2

## (4) Anzahl an unterschiedlichen Werten und deren Verteilung für numerische Werte

In [18]:
num_distribution_pipeline2 = (
    idf
    .assign(total=lambda x: x.groupby(select_num_distribution_pipeline.value)['Risk'].transform('count'))
    .assign(NoRisk=lambda x: x['Risk'].eq(1).groupby(x[select_num_distribution_pipeline.value]).transform('sum') / x['total'] )
    .assign(Risk=lambda x: x['Risk'].eq(0).groupby(x[select_num_distribution_pipeline.value]).transform('sum') / x['total'] )
    .loc[:, [select_num_distribution_pipeline, 'total', 'Risk', 'NoRisk']]
    .sort_values(select_num_distribution_pipeline)
    .drop_duplicates()
)

In [89]:
num_distribution_plot2 = num_distribution_pipeline2.hvplot.line(
    x=select_num_distribution_pipeline,
    y='total',
    title="Anzahl vorkommender Werte in numerischen Spalten",
    ylabel="Anzahl",
    height=500,
    width=600,
    line_width=5,
    legend='top'
)
num_distribution_plot2

## (5) Lineplot der Anzahl an Transaktionen und der Akzeptanzrate pro Tag

In [20]:
df_time.tail(1)

,date,total_number_of_transactions,correct_classifies,share_of_accepted_transactions
30,2023-03-31,510.0,498.0,"0,98"


In [62]:
day_slider = pn.widgets.IntSlider(name='Last Days', start=7, end=31, step=1, value=14)

transactions_pipeline1 = (
    idf2
    .tail(day_slider)
)

In [132]:
transaction_plot1 = transactions_pipeline1.hvplot.line(
    x='date',
    y=['total_number_of_transactions', 'correct_classifies'],
    title="Analyse der korrekt klassifizierten Anträgen",
    ylabel="Anzahl",
    height=500,
    width=600,
    line_width=2,
    color=["#1e85f7", "#04b032"],
    legend='top'
)
transaction_plot1

## (6) kp

In [92]:
transaction_plot2 = transactions_pipeline1.hvplot.line(
    x='date',
    y=['share_of_accepted_transactions'],
    title="Veränderung der Genauigkeit von Klassifizierungen",
    ylabel="Anzahl",
    height=500,
    width=600,
    line_width=2,
    legend='top'
)
transaction_plot2

## (7) Anzahl der Anzahl an Transaktionen usw. als Kennzahl für unten links

In [57]:
import pandas as pd
import panel as pn

# create a slider widget
days_slider = pn.widgets.IntSlider(name='Last Days', start=1, end=30, step=1, value=30)

# create a pipeline to filter the data based on the slider value and calculate the average share of accepted transactions
indicator_pipeline1 = (
    df_time
    .tail(days_slider.value)  # select the last 'days_slider.value' rows
    .assign(average_share=lambda x: x['share_of_accepted_transactions'].mean())  # calculate the average of the 'share_of_accepted_transactions' column
)

# create an indicator widget for the average share of accepted transactions
average_share_indicator = pn.indicators.Number(
    name='Anteil der korrekten Klassifizierungen',
    value=indicator_pipeline1['average_share'].iloc[-1],  # get the last row of the 'average_share' column
    format='{value:.2%}'
)
average_share_indicator

Number(format='{value:.2%}', name='Average Share o..., value=0.9029999999999999)

## Creating Dashboard

In [72]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Machine Learning Monitoring Dashboard', 
    sidebar=[pn.pane.Markdown("# Machine Learning Modell Überwachung Dashboard von Moritz Scheele"), 
             pn.pane.Markdown("#### Das folgende Dashboard wurde im Rahmen der Vorlesung MADS2100 Reporting & Visualisierung gehalten von Dr. Robert Stahlbock entwickelt. Die hier dargestellten Informationen werden auf Basis von fiktiven Daten berechnet. Das Ziel des Dashboards ist es sozial ungerechte Klassifizierungen des Modells zu identifizieren. Dies gelingt in der ersten Zeile, wobei verschiedene Merkmale untersucht werden können. In Zeile zwei kann zusätzlich die Verteilung dieser Merkmale im Datensatz untersucht werden. In der dritten Zeile kann die zeitliche Veränderung der Genauigkeit des Modells eingesehen werden."), 
             average_share_indicator,
             pn.pane.Markdown("### Bekannte Fehler"),
             pn.pane.Markdown("#### Es kann vorkommen, dass beim ersten Aufrufen des Dashboards der Graph zur Betrachtung des Alters in der ersten Zeile nicht geladen wird. Bitte kurz ein anderes Merkmal auswählen, dann sollte es gehen. Zusätzlich werden die Skalen der Visualisierungen nicht automatisch beim Wechsel von Merkmalen zurückgesetzt. Dies kann aber über das Symbol neben den Graphen bewerkstelligt werden."),
             ],
    main=[pn.Row(pn.Column(select_cat_distribution_pipeline, cat_distribution_plot.panel(width=700), margin=(0,25)), 
                 pn.Column(select_num_distribution_pipeline, num_distribution_plot.panel(width=500))), 
          pn.Row(pn.Column(select_cat_distribution_pipeline, cat_distribution_plot2.panel(width=600), margin=(0,25)), 
                 pn.Column(select_num_distribution_pipeline, num_distribution_plot2.panel(width=600))),
         pn.Row(pn.Column(transaction_plot1.panel(width=600), margin=(0,15)), 
                pn.Column(day_slider, margin=(0,15)),
                pn.Column(transaction_plot2.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();